In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


https://www.kaggle.com/datasets/nelgiriyewithana/mcdonalds-store-reviews/code

In [3]:
data = pd.read_csv('/kaggle/input/mcdonalds-store-reviews/McDonald_s_Reviews.csv',encoding='latin')

In [4]:
data

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars


In [5]:
data.isnull().sum()

reviewer_id        0
store_name         0
category           0
store_address      0
latitude         660
longitude        660
rating_count       0
review_time        0
review             0
rating             0
dtype: int64

In [6]:
data = data.dropna(axis=0)

In [7]:
data['rating_count'] = data['rating_count'].apply(lambda x: np.float(re.sub(',','.',x)))

/tmp/ipykernel_3919/1597232591.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['rating_count'] = data['rating_count'].apply(lambda x: np.float(re.sub(',','.',x)))
/tmp/ipykernel_3919/1597232591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating_count'] = data['rating_count'].apply(lambda x: np.float(re.sub(',','.',x)))


In [8]:
#data['review_time'].replace() i think is not true:( (encoding) , onehot is better choice

dummie = pd.get_dummies(data['review_time']).astype(np.float)
data = pd.concat([data,dummie],axis=1)
data=data.drop(['review_time','store_address','store_name','reviewer_id','category'],axis=1)

/tmp/ipykernel_3919/4056589435.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data['review_time']).astype(np.float)


In [9]:
data['rating'] = data['rating'].apply(lambda x: x[0])

In [10]:
stop_words = stopwords.words('english')
def proc_t (names):
    name = names.split()
    name = "".join([word for word in names if word not in stop_words])
    return name

names = data['review'].apply(proc_t)
data = data.drop('review',axis=1)

In [11]:
data = data.astype(np.float)

/tmp/ipykernel_3919/3075087024.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = data.astype(np.float)


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(names)
v_len = len(tokenizer.index_word)
names = tokenizer.texts_to_sequences(names)
v_max = np.max(list(map(lambda x: len(x),names)))

In [16]:
names = pad_sequences(names,maxlen=v_max + 1,padding='post')

In [17]:
names.shape

(32736, 515)

In [19]:
names.shape

(32736, 515)

In [20]:
y = data['rating'].astype(np.int)
y = data['rating'] - 1
x = data.drop('rating',axis=1)

/tmp/ipykernel_3919/997839764.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = data['rating'].astype(np.int)


In [21]:
len(y.unique())
x.shape

5

In [23]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [24]:
names_train , names_test , x_train , x_test , y_train , y_test = train_test_split(names,x,y,train_size=0.7,random_state=13)

In [28]:
input_names = tf.keras.Input(shape=(515 ,))

embedding = tf.keras.layers.Embedding(
    input_dim=v_len,
    input_length=names.shape[1],
    output_dim=64,
)(input_names)

flatten = tf.keras.layers.Flatten()(embedding)

input_other = tf.keras.Input(shape=(42,))

layer_1 = tf.keras.layers.Dense(64, activation='relu')(input_other)
layer_2 = tf.keras.layers.Dense(64, activation='relu')(layer_1)

concat = tf.keras.layers.concatenate([flatten, layer_2])

outputs = tf.keras.layers.Dense(5, activation='softmax')(concat)

model = tf.keras.Model(inputs=[input_names, input_other], outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
with tf.device('GPU:0'):
    hist = model.fit(
        [names_train, x_train],
        y_train,
        validation_split=0.2,
        batch_size=10,
        epochs=10,
        callbacks=tf.keras.callbacks.ReduceLROnPlateau()
    )

Epoch 1/10
1834/1834 [==============================] - 52s 27ms/step - loss: 1.0580 - accuracy: 0.5852 - val_loss: 0.9494 - val_accuracy: 0.6288 - lr: 0.0010
Epoch 2/10
1834/1834 [==============================] - 13s 7ms/step - loss: 0.7663 - accuracy: 0.7098 - val_loss: 0.9270 - val_accuracy: 0.6452 - lr: 0.0010
Epoch 3/10
1834/1834 [==============================] - 10s 6ms/step - loss: 0.5825 - accuracy: 0.7924 - val_loss: 0.9518 - val_accuracy: 0.6583 - lr: 0.0010
Epoch 4/10
1834/1834 [==============================] - 10s 5ms/step - loss: 0.4518 - accuracy: 0.8439 - val_loss: 1.0202 - val_accuracy: 0.6587 - lr: 0.0010
Epoch 5/10
1834/1834 [==============================] - 9s 5ms/step - loss: 0.3616 - accuracy: 0.8786 - val_loss: 1.0746 - val_accuracy: 0.6472 - lr: 0.0010
Epoch 6/10
1834/1834 [==============================] - 9s 5ms/step - loss: 0.3004 - accuracy: 0.8964 - val_loss: 1.1797 - val_accuracy: 0.6494 - lr: 0.0010
Epoch 7/10
1834/1834 [==============================]

In [29]:
model.evaluate([names_test,x_test],y_test)

307/307 [==============================] - 1s 2ms/step - loss: 1.6357 - accuracy: 0.6375


[1.6356782913208008, 0.6375114321708679]